In [29]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema


In [36]:
# Initialize parameters
window = 100
gain_window = 5
extractor = ALPClientExtractor(os.getenv("APCAKEY"), os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
tickers = [x + "/USD" for x in tickers]

prices = []
start = datetime.now() - timedelta(days=465)
end = datetime.now()

# Loop through each ticker to fetch prices and calculate metrics
for ticker in tickers:
    response = extractor.crypto(ticker, start,end)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c": "adjclose", "t": "date"})[["date", "adjclose"]]
    
    # Convert date and sort
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    
    # Calculate metrics
    price["rolling_return"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"]) / price["adjclose"])
    price["bollinger"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"].rolling(window).std() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(gain_window)
    price["avg_return"] = price["return"].pct_change(gain_window).rolling(window).mean()
    price["coev"] = (price["adjclose"].rolling(window).mean() / price["adjclose"].rolling(window).std())
    price["ticker"] = ticker
    prices.append(price)

# Combine all prices into a single DataFrame
sim = pd.concat(prices).sort_values("date").dropna()

In [37]:
sim

,date,adjclose,rolling_return,bollinger,return,avg_return,coev,ticker
109,2023-10-07 05:00:00+00:00,0.061403,0.092211,-0.002493,-0.013737,-1.210031,11.532898,DOGE/USD
109,2023-10-07 05:00:00+00:00,0.085362,0.168834,0.010689,-0.040629,-4.009258,7.390911,GRT/USD
109,2023-10-07 05:00:00+00:00,65.802550,0.180425,-0.047001,-0.007054,-0.391555,5.190373,LTC/USD
109,2023-10-07 05:00:00+00:00,10.396300,0.093177,-0.082398,0.116888,-0.531450,6.226272,AVAX/USD
109,2023-10-07 05:00:00+00:00,0.171892,0.090047,-0.009205,-0.037020,-1.245174,10.982632,BAT/USD
...,...,...,...,...,...,...,...,...
465,2024-09-27 05:00:00+00:00,7.416655,-0.024405,-0.176691,0.092323,3.340592,6.406315,UNI/USD
465,2024-09-27 05:00:00+00:00,29.901500,-0.160314,-0.256737,0.082501,-2.149310,8.708356,AVAX/USD
406,2024-09-27 05:00:00+00:00,0.751620,-0.054665,-0.136312,0.083632,-1.345835,11.578394,XTZ/USD
465,2024-09-27 05:00:00+00:00,1676.150500,0.272073,0.004722,0.035094,-1.505371,4.758066,MKR/USD


In [38]:
metrics = [
    "rolling_return",
    "bollinger",
    "return",
    "avg_return",
    "coev"
]

In [41]:
import copy
analysis = []
date_range = list(sim["date"].unique())
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
for metric in metrics:
    for ascending in [True,False]:
        cash = 100
        position = copy.deepcopy(base_position)
        cashes = []
        for date in tqdm(date_range):
            try:
                position = copy.deepcopy(position)
                position["date"] = date
                today = sim[sim["date"]==date]
                if today.index.size >= 0:
                    opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        pnl_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] >= 0.05)
                        loss_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] <= -0.05)
                        hpr_req = (date - position["buy_date"]).days > 7
                        if hpr_req:
                            try:
                                cash = position["adjclose"] * position["quantity"] * 0.99
                                cashes.append(cash)
                                position = copy.deepcopy(base_position)
                                position["date"] = date
                                position["ticker"] = opportunity["ticker"]
                                position["quantity"] = cash / opportunity["adjclose"] 
                                position["adjclose"] = opportunity["adjclose"]
                                position["buy_price"] = opportunity["adjclose"]
                                position["buy_date"] = date
                            except Exception as e:
                                continue
                    else:
                        position["date"] = date
                        position["ticker"] = opportunity["ticker"]
                        position["quantity"] = cash / opportunity["adjclose"] 
                        position["adjclose"] = opportunity["adjclose"]
                        position["buy_price"] = opportunity["adjclose"]
                        position["buy_date"] = date
            except Exception as e:
                continue
        position["metric"] = metric
        position["ascending"] = ascending
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(copy.deepcopy(position))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 357/357 [00:00<00:00, 1256.76it/s]


In [42]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,pv
1,2024-09-27 05:00:00+00:00,MKR/USD,0.122500,1676.15050,1567.90900,2024-09-24 05:00:00+00:00,rolling_return,False,205.328292
4,2024-09-27 05:00:00+00:00,AVAX/USD,6.079354,29.90150,27.61975,2024-09-24 05:00:00+00:00,return,True,181.781817
3,2024-09-27 05:00:00+00:00,MKR/USD,0.106103,1676.15050,1567.90900,2024-09-24 05:00:00+00:00,bollinger,False,177.844397
8,2024-09-27 05:00:00+00:00,AAVE/USD,0.926517,167.16450,171.51900,2024-09-24 05:00:00+00:00,coev,True,154.880803
2,2024-09-27 05:00:00+00:00,AAVE/USD,0.723285,167.16450,171.51900,2024-09-24 05:00:00+00:00,bollinger,True,120.907601
9,2024-09-27 05:00:00+00:00,BTC/USD,0.001787,65803.36900,64177.63100,2024-09-24 05:00:00+00:00,coev,False,117.585946
7,2024-09-27 05:00:00+00:00,BTC/USD,0.001334,65803.36900,64177.63100,2024-09-24 05:00:00+00:00,avg_return,False,87.781373
0,2024-09-27 05:00:00+00:00,AAVE/USD,0.491278,167.16450,171.51900,2024-09-24 05:00:00+00:00,rolling_return,True,82.124263
6,2024-09-27 05:00:00+00:00,AAVE/USD,0.394241,167.16450,171.51900,2024-09-24 05:00:00+00:00,avg_return,True,65.903061
5,2024-09-27 05:00:00+00:00,GRT/USD,84.528387,0.19015,0.18244,2024-09-24 05:00:00+00:00,return,False,16.073073
